<img style="float:right" src="images/logo_va.png" /> 

# Visione Artificiale
## Esercitazione: Analisi di immagini binarie

### Sommario
* Segmentazione oggetti dal background per ottenere una maschera foreground/background
* Utilizzo di tecniche di morfologia matematica per migliorare la maschera
* Etichettatura delle componenti connesse per rimuovere piccoli "buchi" dal foreground
* Etichettatura delle componenti connesse e loro area
* Estrazione del contorno di ogni componente connessa e calcolo della circonferenza minima che lo contiene
* Selezione delle componenti connesse di forma circolare

Iniziamo con l'importazione dei moduli che ci serviranno: `NumPy`, `OpenCV`, `va`. Importiamo anche la funzione `interact` di Jupyter.

In [ ]:
import numpy as np
import cv2 as cv
import va
from ipywidgets import interact

Consideriamo l'immagine seguente in cui vogliamo separare gli oggetti dallo sfondo.

In [ ]:
img = cv.imread('analisi/pralines.png')
va.show(img)

Non è un caso particolarmente difficile in quanto lo sfondo è piuttosto omogeneo, tuttavia, come si può verificare nella cella seguente, una semplice conversione in grayscale e binarizzazione con soglia globale non è sufficiente, per la presenza di alcuni oggetti più chiari di alcune parti dello sfondo. 

In [ ]:
img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

@interact(soglia=(0,255))
def binarizzazione_globale(soglia=128):
    va.show(cv.threshold(img_gray, soglia, 255, cv.THRESH_BINARY)[1])

<img style="float:left" src="images/ar.png" />**Esercizio 1** - Trovare un metodo per separare i cioccolatini dell'immagine `img` dallo sfondo. La variabile `mask` deve fare riferimento a un'immagine di byte con solo due valori: 255 (foreground) e 0 (background).  
Suggerimenti: un possibile metodo consiste nel convertire l'immagine in HSV e utilizzare: 1) il canale della luminosità per trovare i cioccolatini scuri (con una semplice binarizzazione con soglia globale); 2) il canale hue per individuare i cioccolatini chiari (trovando il particolare range di valori di hue che li caratterizza). La funzione `cv.inRange` può essere utile per costruire delle maschere a partire da range di valori hsv, anche se non è indispensabile utilizzarla.  
Come sempre, si possono aggiungere tutte le celle che si desidera per sperimentare i vari metodi: l'importante è che venga inizializzata la variabile `mask` con il risultato.

In [ ]:
# --- Svolgimento Esercizio 1: Inizio --- #

_, mask = cv.threshold(img_gray, 128, 255, cv.THRESH_BINARY) # Da sostituire con la soluzione
 
# --- Svolgimento Esercizio 1: Fine --- #    

<img style="float:left" src="images/in.png" />L'esecuzione della cella seguente mostra il risultato ottenuto a fianco di un possibile risultato corretto. Si noti che il risultato non deve essere esattamente quello suggerito: è sufficiente che non sia troppo differente. Come si può notare il risultato non deve essere "perfetto": è normale che ci siano alcuni "buchi" all'interno del foreground e piccole parti del background incorrettamente considerate forground. Nel seguito cercheremo di migliorare la maschera.

In [ ]:
mask_esempio = cv.imread('analisi/mask_esempio.png', cv.IMREAD_GRAYSCALE)
va.show((mask,'Risultato ottenuto (mask)'), (mask_esempio,'Esempio di risultato corretto'))
def am(m): t = img.copy(); t[m==0]//=4; return t
va.show((am(mask),'Segmentazione con mask'), (am(mask_esempio),"Segmentazione con l'esempio di risultato corretto"))

<img style="float:left" src="images/ar.png" />**Esercizio 2** - Osservare il risultato ottenuto nell'esercizio precedente (`mask`): è probabile che nella mascera siano presenti alcuni o tutti i seguenti problemi:

1. alcuni cioccolatini non sono ben "separati", ossia costituiscono un'unica componente connessa;
2. alcuni cioccolatini presentano dei "buchi" o delle "concavità", ossia pixel incorrettamente assegnati al background;
3. alcuni pixel del background sono incorrettamente assegnati al foreground.

Utilizzare tecniche di morfologia matematica per cercare di risolvere tali problemi. In particolare si suggerisce di utilizzare un'operazione di *chiusura* per ridurre il secondo e terzo problema, seguita da un'operazione di *apertura* per il primo problema. La variabile `mask1` deve fare riferimento alla maschera risultante.  
**Nota bene**: è importante che al termine di questo esercizio tutti i cioccolatini siano separati e non ci siano componenti connesse spurie. In altre parole devono essere completamente risolti i problemi 1 e 3. Il problema 2 può essere ancora presente, purchè i pixel di background all'interno dei cioccolatini siano "buchi": devono essere completamente circondati da pixel di foreground. Quest'ultimo problema, se presente, sarà affrontato nell'esercizio seguente.

In [ ]:
# --- Svolgimento Esercizio 2: Inizio --- #

mask1 = mask.copy() # Da sostituire con la soluzione

# --- Svolgimento Esercizio 2: Fine --- #   

<img style="float:left" src="images/in.png" />L'esecuzione della cella seguente mostra il risultato ottenuto a fianco di un possibile risultato corretto. Si noti che il risultato non deve essere esattamente quello suggerito: è sufficiente che ci sia una componente connessa per ogni cioccolatino, con eventualmente qualche piccolo "buco".

In [ ]:
mask1_esempio = cv.imread('analisi/mask1_esempio.png', cv.IMREAD_GRAYSCALE)
va.show((mask1,'Risultato ottenuto (mask1)'), (mask1_esempio,'Esempio di risultato corretto'))
va.show((am(mask1),'Segmentazione con mask1'), (am(mask1_esempio),"Segmentazione con l'esempio di risultato corretto"))

Eseguire la cella seguente per verificare che `mask1` contenga il numero atteso di componenti connesse.

In [ ]:
va.test_analisi_2(mask1)

<img style="float:left" src="images/ar.png" />**Esercizio 3** - Costruire una maschera `mask2` a partire da `mask1`, chiudendo eventuali buchi ancora presenti all'interno delle componenti connesse di `mask1`. Se le componenti connesse sono già prive di "buchi" non è necessario eseguire questo esercizio e si può semplicemente copiare `mask1` su `mask2`.  
Suggerimento: un modo semplice per individuare i pixel dei "buchi" è eseguire l'algoritmo dell'etichettatura delle componenti connesse sul negativo di `mask1` (ossia scambiando il background con il foreground). Le componenti connesse trovate saranno il vero background e tante piccole componenti corrispondenti ai "buchi": si possono chiudere i "buchi" assegnando i pixel appartenenti alle componenti connesse con area minore di una certa soglia al foreground.

In [ ]:
# --- Svolgimento Esercizio 3: Inizio --- #

mask2 = mask1.copy() # Da sostituire con la soluzione

# --- Svolgimento Esercizio 3: Fine --- #

<img style="float:left" src="images/in.png" />L'esecuzione della cella seguente mostra il risultato ottenuto a fianco di un possibile risultato corretto.

In [ ]:
mask2_esempio = cv.imread('analisi/mask2_esempio.png', cv.IMREAD_GRAYSCALE)
va.show((mask2,'Risultato ottenuto (mask2)'), (mask2_esempio,'Esempio di risultato corretto'))
va.show((am(mask2),'Segmentazione con mask2'), (am(mask2_esempio),"Segmentazione con l'esempio di risultato corretto"))

Eseguire la cella seguente per verificare che `mask2` contenga il numero atteso di componenti connesse.

In [ ]:
va.test_analisi_3(mask2)

<img style="float:left" src="images/ar.png" />**Esercizio 4** - Eseguire l'algoritmo di etichettatura delle componenti connesse su `mask2` e calcolare anche l'area (numero di pixel) di ciascuna componente connessa. Inizializzare una lista `cca` contenente, per ciascuna componente connessa (escludendo il background), una tupla `(m, a)` dove `m` è un'immagine binaria con le stesse dimensioni di `mask2` con tutti i pixel a zero tranne quelli della componente connessa (che devono essere pari a 255), mentre `a` è la sua area.

In [ ]:
# --- Svolgimento Esercizio 4: Inizio --- #

cca = [(mask2.copy(), 0)] # Da sostituire con la soluzione

# --- Svolgimento Esercizio 4: Fine --- #

<img style="float:left" src="images/in.png" />Eseguire la cella seguente per verificare che `cca` contenga il numero atteso di tuple con il formato corretto.

In [ ]:
va.test_analisi_4(cca,mask2)

<img style="float:left" src="images/in.png" />L'esecuzione della cella seguente consente di scorrere la lista `cca` visualizzando il contenuto di ciascun elemento sull'immagine.

In [ ]:
@interact(index=(0,len(cca)-1))
def mostra_cca(index=0):
    tmp = img.copy()
    m, a = cca[index]
    tmp[m==255] = (0,255,0)
    x, y, w, h = cv.boundingRect(m)    
    va.center_text(tmp, f'Area={a}', (x+w//2, y+h//2), (255,0,0))
    va.show(tmp)

<img style="float:left" src="images/ar.png" />**Esercizio 5** - A partire dalla lista `cca`, costruire una nuova lista `info` che contenga una tupla `(m, c, a, ((xc, yc), r))` per ciascun elemento di `cca`, dove:

* `m` è la stessa maschera contenuta nel corrispondente elemento di `cca`, ossia la maschera della corrispondente componente connessa;
* `c` è il contorno della componente connessa, ottenuto mediante `cv.findContours()`;
* `a` è l'area della componente connessa;
* `((xc, yc), r)` è una tupla contenente le coordinate `(xc, yc)` e il raggio `r` del minimo cerchio che racchiude tutti i pixel del contorno `c`.

Suggerimenti: chiamando `cv.findContours()` su ciascuna componente connessa si possono ottenere i contorni. Prestare attenzione al fatto che `cv.findContours()` restituisce una lista di contorni, ma in questo caso sappiamo che ce ne sarà soltanto uno, per cui `c` sarà semplicemente il primo contorno trovato. `cv.minEnclosingCircle()` permette di trovare `((xc, yc), r)` a partire dal contorno `c`.

In [ ]:
# --- Svolgimento Esercizio 5: Inizio --- #

info = [(mask2.copy(), [np.array(((0,0),(1,1)))], 0, ((100,100), 42))] # Da sostituire con la soluzione

# --- Svolgimento Esercizio 5: Fine --- #

<img style="float:left" src="images/in.png" />L'esecuzione della cella seguente consente di scorrere la lista `info` visualizzando il contenuto di ciascun elemento sull'immagine.

In [ ]:
def ir(x): return int(round(x))

@interact(i=(0,len(info)-1))
def show_info2(i=0):
    m, c, a, ((xc,yc), r) = info[i]
    res = img.copy()
    res[m==0]//=8   
    cv.drawContours(res, c, -1, (0,255,255), 2, cv.LINE_AA)
    cv.circle(res, (ir(xc), ir(yc)), ir(r), (0,128,0), 2, cv.LINE_AA)    
    va.center_text(res, f'{a}', (ir(xc), ir(yc)), (255,255,255))
    va.show(res)

<img style="float:left" src="images/ar.png" />**Esercizio 6** - A partire dalla lista `info`, costruire una nuova lista `tondi` che contenga i soli elementi di `info` che corrispondono a cioccolatini di forma circolare.  
Suggerimento: uno dei modi più semplici consiste nel considerare la differenza fra l'area del cerchio circoscritto e l'area effettiva della componente connessa.

In [ ]:
# --- Svolgimento Esercizio 6: Inizio --- #

tondi = [info[0], info[1], info[2]] # Ovviamente non si possono selezionare "manualmente", trovare un algoritmo!

# --- Svolgimento Esercizio 6: Fine --- #

<img style="float:left" src="images/in.png" />L'esecuzione della cella seguente mostra il risultato ottenuto a fianco del risultato atteso.

In [ ]:
res = img.copy()
for _, c, *_ in tondi:
    cv.drawContours(res, [c], -1, (255,128,0), 2, cv.LINE_AA)
va.show((res,'Risultato ottenuto'), (cv.imread('analisi/res.png'),'Risultato atteso'))